In [619]:
import numpy as np
import pandas as pd
import re
import nltk
import difflib
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from nltk.stem.porter import *

In [620]:
data = pd.read_csv('nlp project.csv', encoding = 'utf-8').fillna(' ')

In [621]:
new_attribute_value = []
for index in data.attribute_value:
    index = re.sub(r'\s','', index)
    new_attribute_value.append(index)
data['new_attribute_value'] = new_attribute_value

In [622]:
data1 = data[['brand','brand_category','description','product_full_name','details','attribute_name', 'new_attribute_value', 'file']]

In [623]:
data1=data1.drop_duplicates()
len(data1)

68326

In [624]:
data=data1

# Description Feature Pre-Processing (Regex)

In [625]:
nosymbol_description = []
for sent in data.description:
    nosymbol_description.append(re.sub(r'[^\w+0-9%.]', ' ', sent, flags = re.IGNORECASE))

In [626]:
all_m = pd.Series(nosymbol_description).str.findall(r'(\d+)(?:%)\s(\w+)')
material = []
temp_content = ''
temp = []
number = 0
for i in range(len(all_m)):
    if all_m[i] != []:
        for j in all_m[i]:
            for k in j:
                if k == j[0]:
                    number = int(k)
                    if number >= 80:
                        temp_content = temp_content + "HIGH_"
                    elif number >= 40:
                        temp_content = temp_content + "MEDIAN_"
                    else:
                        temp_content = temp_content + "LOW_"
                if k == j[-1]:
                    temp_content = temp_content + k.upper()
            
            temp.append(temp_content)
            temp_content = ''
        material.append(temp)
        temp = []
    else:
         material.append('')

In [627]:
material[3]

['HIGH_PIMA', 'LOW_ELASTANE']

In [628]:
material_description = []
for i in data.description:
    material_description.append(re.sub(r'(\d+)(?:%)\s(\w+)', 'MATERIAL', i, flags = re.IGNORECASE))

In [629]:
count = 0
cleaned_material = []
for i in range(len(material_description)):
    new_word = []
    for j in material_description[i].split(' '):
        if j == 'MATERIAL':
            new_word.append(material[i][count])
            count += 1
        else:
            new_word.append(j.lower())
    count = 0
    clean = " ".join(new_word)
    cleaned_material.append(clean)
        
            
            

cleaned_material

In [631]:
no_numeric_description = []
for i in cleaned_material:
    no_numeric_description.append(re.sub(r'((\d+\.\d+)|(\d+))', 'NUMERICAL_VALUE ', i, flags = re.IGNORECASE))

no_numeric_description

In [633]:
def word_count(lines, delimiter = " "):
    words = Counter()
    for line in lines:
        line = line.lower()
        for word in line.split(delimiter):
            words[word] += 1
    return words

In [634]:
count = word_count(no_numeric_description)
count_sort = sorted((value, key) for (key, value) in count.items())[::-1]

In [ ]:

stopword = set(stopwords.words('english'))
count_sorted_set = set(pd.DataFrame(count_sort[:500])[1])
count_sorted_set.difference_update(stopword)
count_sorted_set

In [636]:
updated_stopwords = stopwords.words('english') + [
    '.' , '', 
]

In [ ]:
no_numeric_description

In [638]:
cleaned_descriptions = []
updated_stopwords = set(updated_stopwords)

for line in no_numeric_description:
    new_words = []
    line = re.findall(r'\b[a-zA-Z0-9_]{2,}\b',line)
    for word in line:
        if word.lower() in updated_stopwords:
            continue
        new_words.append(word)
    cleaned_description = " ".join(new_words)
    cleaned_descriptions.append(cleaned_description)
    

In [ ]:
cleaned_descriptions

# Brand Feature Pre-Processing (Regex)

In [640]:
def removeSpecialChars(z):
    
    return z.translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./'<>?\|`~-=_+"})

def initial_clean(data,column):
    
    data[column]=data[column].apply(str)
    data[column]=data[column].apply(lambda x:x.lower())
    data[column]=data[column].apply(lambda x:re.sub("[\(\[].*?[\)\]]", "", x))
    data[column]=data[column].apply(lambda x:removeSpecialChars(x))
    
initial_clean(data1,'brand')

In [641]:
brand = data1['brand'].fillna(" ")
stopwords = ['by','and','the','of','for','to','inc','','a','an']
def remove_stopwords(text: str):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = list(filter(lambda token: token not in stopwords, tokens))
    return " ".join(filtered_tokens)
brand = brand.apply(remove_stopwords)

In [ ]:
brand

# Details Feature Pre-Processsing (Regex)

In [643]:
#get required features and fill in null values with blank
df1=data1["details"]
df1=df1.fillna(" ")

#extract all % number in details as a new column
perc=df1.str.findall(r"(\d+)(?:%)\s(\w+)")
result=[]
temp_result=[]
temp=""
for i in range(len(perc)):
    if perc[i]!=[]:
        for j in range(len(perc[i])):
            if int(perc[i][j][0])>=80:
                temp="High_"+perc[i][j][1]
            elif int(perc[i][j][0])>=60:
                temp="Medium_"+perc[i][j][1]
            else:
                temp="Low_"+perc[i][j][1]
            temp_result.append(temp)
        result.append(temp_result)
        temp=""
        temp_result=[]
    else:
        result.append("")

In [ ]:
#remove punctuations and numbers
df1=df1.apply(lambda x: x.lower()).str.replace(r"[^a-z\s]+", "", case=False)
df1=df1.str.replace(r"\r\n", " ", case=False)
df1

In [645]:
count=word_count(list(df1))
count_sort=sorted((value, key) for (key, value) in count.items())[::-1]

In [646]:
from nltk.corpus import stopwords
nltk.download("stopwords")
# set(stopwords.words("English"))
nltk_stopwords = list(set(stopwords.words("English")))
count_sorted_set = set(pd.DataFrame(count_sort[:100])[1])
count_sorted_set.difference_update(nltk_stopwords)
nltk_stopwords.extend(["","h","l","mm","w","x","xl","xs","cm","uv"])

def remove_stopwords(title: str):
    tokens = nltk.word_tokenize(title)
    filtered_tokens = list(filter(lambda token: token not in nltk_stopwords, tokens))
    return " ".join(filtered_tokens)

[nltk_data] Downloading package stopwords to C:\Users\Sharon
[nltk_data]     Jia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

#check spelling and remove stopwords
#df1["details"]=df1["details"].apply(lambda x: spellcheck_document(x))
df1=df1.apply(lambda x: remove_stopwords(x))
df1

# Product Name Pre-Processing (Regex)

In [648]:
# Make everything lowercase
product_full_name = data1['product_full_name'].str.lower()
# Remove hyphen
product_full_name = product_full_name.str.replace("-","",case=False)
# Remove numbers
product_full_name = product_full_name.str.replace(r'[0-9]+',"",case=False)
# Remove special characters and stop words
import nltk
stopwords = ['&','|',',','.',"'",'in','le','mm','and','with','the','k','x','les','de','of',"'s"]
def remove_stopwords(text: str):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = list(filter(lambda token: token not in stopwords, tokens))
    return " ".join(filtered_tokens)
product_full_name = product_full_name.apply(remove_stopwords)

# Lemmatization

In [650]:
lemmatizer = WordNetLemmatizer()

#function to convert nltk tag to wordnet tag
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    wn_tagged = map(lambda x :(x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word,tag))
    return " ".join(res_words)

In [651]:
cleaned_description_lem = []
for sentence in cleaned_descriptions:
    cleaned_description_lem = cleaned_description_lem + [lemmatize_sentence(sentence)]

In [653]:
cleaned_brand_lem = []
for sentence in list(brand):
    cleaned_brand_lem = cleaned_brand_lem + [lemmatize_sentence(sentence)]

In [654]:
cleaned_details_lem = []
for sentence in list(df1):
    cleaned_details_lem = cleaned_details_lem + [lemmatize_sentence(sentence)]

In [655]:
cleaned_productname_lem = []
for sentence in list(product_full_name):
    cleaned_productname_lem = cleaned_productname_lem + [lemmatize_sentence(sentence)]

# Stemming

In [657]:
stemmer = PorterStemmer()
def stemm_sentence(sentence):
    word_token = nltk.word_tokenize(sentence)
    res_words = []
    for word in word_token:
        res_words.append(stemmer.stem(word))
    return " ".join(res_words)

In [658]:
cleaned_description_stemm = []
for sentence in cleaned_descriptions:
    cleaned_description_stemm = cleaned_description_stemm + [stemm_sentence(sentence)]

In [659]:
cleaned_brand_stemm = []
for sentence in list(brand):
    cleaned_brand_stemm = cleaned_brand_stemm + [stemm_sentence(sentence)]

In [660]:
cleaned_details_stemm = []
for sentence in list(df1):
    cleaned_details_stemm = cleaned_details_stemm + [stemm_sentence(sentence)]

In [661]:
cleaned_productname_stemm = []
for sentence in list(product_full_name):
    cleaned_productname_stemm = cleaned_productname_stemm + [stemm_sentence(sentence)]

# TF-IDF Score

## Description Feature

In [662]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),
                             token_pattern=r'\b[a-zA-Z0-9]{2,}\b',
                             max_df=0.8,
                             min_df = 0.01, stop_words=updated_stopwords, max_features = 500)

In [663]:
X = vectorizer.fit_transform(cleaned_description_lem)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
description_lem_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
description_lem_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

In [664]:
X = vectorizer.fit_transform(cleaned_description_stemm)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
description_stemm_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
description_stemm_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

## Brand Feature

In [665]:
X = vectorizer.fit_transform(cleaned_brand_lem)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
brand_lem_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
brand_lem_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

In [666]:
X = vectorizer.fit_transform(cleaned_brand_stemm)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
brand_stemm_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
brand_stemm_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

## Details Feature

In [667]:
X = vectorizer.fit_transform(cleaned_details_lem)
terms = vectorizer.get_feature_names() 
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
details_lem_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
details_lem_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

In [668]:
X = vectorizer.fit_transform(cleaned_details_stemm)
terms = vectorizer.get_feature_names() 
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
details_stemm_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
details_stemm_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

## Product Name Feature

In [669]:
X = vectorizer.fit_transform(cleaned_productname_lem)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
productname_lem_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
productname_lem_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

In [670]:
X = vectorizer.fit_transform(cleaned_productname_stemm)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
productname_stemm_tfidf_score = pd.DataFrame(tf_idf, columns=["score"])
productname_stemm_tfidf_score.sort_values(by="score", ascending=False, inplace=True)

# Word2Vec

## Description Feature

In [671]:
docs = [word_tokenize(sentence) for sentence in cleaned_description_lem ]
description_lem_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

In [672]:
docs = [word_tokenize(sentence) for sentence in cleaned_description_stemm]
description_stemm_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

## Brand Feature

In [673]:
docs = [word_tokenize(sentence) for sentence in cleaned_brand_lem ]
brand_lem_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

In [674]:
docs = [word_tokenize(sentence) for sentence in cleaned_brand_stemm ]
brand_stemm_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

## Details Feature

In [675]:
docs = [word_tokenize(sentence) for sentence in cleaned_details_lem ]
details_lem_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

In [676]:
docs = [word_tokenize(sentence) for sentence in cleaned_details_stemm ]
details_stemm_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

## Product Name Feature

In [677]:
docs = [word_tokenize(sentence) for sentence in cleaned_productname_lem ]
productname_lem_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

In [678]:
docs = [word_tokenize(sentence) for sentence in cleaned_productname_stemm ]
productname_stemm_word2vec_model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

In [679]:
product = pd.Series(cleaned_description_lem)+" "+pd.Series(cleaned_brand_lem)+" "+\
        pd.Series(cleaned_details_lem)+" "+pd.Series(cleaned_productname_lem)
product = list(product)

In [680]:
phrases = Phrases(product, min_count=10)
bi_gram = Phraser(phrases)
product=list(bi_gram[product])

In [681]:
docs = [word_tokenize(sentence) for sentence in product]
model = Word2Vec(docs, min_count = 1, window = 5, size = 200)

In [682]:
product_vector = []
for item in product:
    temp_vector = np.zeros((1,200))
    n = 0
    for word in item.split(" "):
        if word in model.wv.vocab:
            temp_vector += model.wv.get_vector(word)
        else:
            temp_vector += np.random.normal(size=200)
        n += 1
    product_vector.append(temp_vector/n)

# RNN

In [909]:
def pred_category(data,threshold):
    pred = list(map(lambda v:[i for i, e in enumerate(v) if e > threshold] ,data))
    return np.array(pred)

def pred_category_name(data,threshold):
    pred = list(map(lambda v:[list(dummies.columns)[i] for i, e in enumerate(v) if e > threshold] ,data))
    return np.array(pred)

def actual_category(data):
    actual=list()
    for v in data:
        t = []
        for i,a in enumerate(v):
            if a == 1:
                t.append(i)
        actual.append(t)
    return actual

def actual_category_name(data):
    actual=list()
    for v in data:
        t = []
        for i,a in enumerate(v):
            if a == 1:
                t.append(list(dummies.columns)[i])
        actual.append(t)
    return actual

def F1(pred_result,actual_result):
    def recall(pred_result,actual_result,attribute_value):   
        Actual=0
        TP=0

        for i in range(len(actual_result)):
            if actual_result[i]==attribute_value:
                Actual+= 1

                pred = set(pred_result[i])
                true = set(actual_result[i])

                if len(pred.intersection(true)) != 0:
                    TP += 1

        return TP/Actual if Actual>0 else ''


    def Precision(pred_result,actual_result,attribute_value):   
        Positive=0
        TP=0

        for i in range(len(pred_result)):
            if pred_result[i]==attribute_value:
                Positive+= 1

                pred = set(pred_result[i])
                true = set(actual_result[i])

                if len(pred.intersection(true)) != 0:
                    TP += 1
                    
        return TP/Positive if Positive>0 else ''

    F1=pd.DataFrame(list(dummies.columns))
    F1['attribute_name']=attribute
    F1.columns=['attribute_value','attribute_name']
    F1=F1[['attribute_name','attribute_value']]
    F1['recall']=''
    F1['Precision']=''
    F1['F1']=''
    for i in F1.index:
        try:
            F1.loc[i,'recall']=  recall(pred_result,actual_result,[F1.loc[i,'attribute_value']])
        except:
            continue
    for i in F1.index:
        try:
            F1.loc[i,'Precision']=Precision(pred_result,actual_result,[F1.loc[i,'attribute_value']])
        except:
            continue
            
    for i in F1.index:
        try:
            F1.loc[i,'F1']=2*(F1.loc[i,'recall']*F1.loc[i,'Precision'])/(F1.loc[i,'recall']+F1.loc[i,'Precision'])
        except:
            continue  

    return F1

## input - product_vector

In [1163]:
df = pd.DataFrame({ 'input':pd.Series(product_vector),
                    'attribute_name': list(data1['attribute_name']),
                    'attribute_value': list(data1['new_attribute_value'])
                   })

df['attribute_value']=df['attribute_value'].apply(lambda x:x.lower())

In [1157]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, LSTM
from keras.layers import Embedding
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.preprocessing.text import text_to_word_sequence
from typing import List

## all_attributes

In [ ]:
df=df[~df['attribute_value'].isna()]
dummies = pd.get_dummies(df['attribute_value'])
data = pd.concat([df, dummies], axis=1, sort=False)
data=data.drop('attribute_value',axis=1)
data=data.drop('attribute_name',axis=1)

X=data['input']
y= data.iloc[:,1:].values

X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.25,random_state=1)

vocab_size = 200

from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

max_length = 200

X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# define rnn model

model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
model.add(Dense(y_train.shape[1], activation='softmax'))

from keras.utils.vis_utils import plot_model

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
# print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)

# fit the model
model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)

pred_vectors_train = model.predict(X_train)
pred_vectors_test = model.predict(X_test)

model.save('product_vector_model.h5')

In [ ]:
model.save('product_vector_model.h5')

In [ ]:
df=df[~df['attribute_value'].isna()]
dummies = pd.get_dummies(df['attribute_value'])
data = pd.concat([df, dummies], axis=1, sort=False)
data=data.drop('attribute_value',axis=1)
data=data.drop('attribute_name',axis=1)

X=data['input']
y= data.iloc[:,1:].values

X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.25,random_state=1)

vocab_size = 200

from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

max_length = 200

X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# define rnn model

model = Sequential()
model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
model.add(Dense(100,activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

from keras.utils.vis_utils import plot_model

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
# print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)

# fit the model
model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)

pred_vectors_train = model.predict(X_train)
pred_vectors_test = model.predict(X_test)

model.save('product_vector_model.h5')

## style

In [1166]:
attribute='style'
data=df[df['attribute_name'] == attribute]
dummies = pd.get_dummies(data['attribute_value'])
data = pd.concat([data, dummies], axis=1, sort=False)
data=data.drop('attribute_value',axis=1)
data=data.drop('attribute_name',axis=1)

X=data['input']
y= data.iloc[:,1:].values

X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.25,random_state=1)

vocab_size = 200

from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

max_length = 200

X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# define rnn model

model = Sequential()
model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
model.add(Dense(100,activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

from keras.utils.vis_utils import plot_model

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
# print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)

# fit the model
model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)

style_pred_vectors_train = model.predict(X_train)
style_pred_vectors_test = model.predict(X_test)

## save model

# model.save('style_combine_stemm_tfidf_score_model.h5')
model.save('style_product_vector_model.h5')

In [1170]:
threshold=0.1
style_train_pred_classes = pred_category(style_pred_vectors_train,threshold)
style_test_pred_classes = pred_category(style_pred_vectors_test,threshold)

style_train_pred_classes_name = pred_category_name(style_pred_vectors_train,threshold)
style_test_pred_classes_name = pred_category_name(style_pred_vectors_test,threshold)

style_train_true_classes = actual_category(y_train)
style_test_true_classes = actual_category(y_test)

style_train_true_classes_name = actual_category_name(y_train)
style_test_true_classes_name = actual_category_name(y_test)

style_test_F1=F1(style_test_pred_classes_name,style_test_true_classes_name)
style_train_F1=F1(style_train_pred_classes_name,style_train_true_classes_name)

In [ ]:
style_test_F1

In [ ]:
style_train_F1

## occasion

In [917]:
attribute='occasion'
data=df[df['attribute_name'] == attribute]
dummies = pd.get_dummies(data['attribute_value'])
data = pd.concat([data, dummies], axis=1, sort=False)
data=data.drop('attribute_value',axis=1)
data=data.drop('attribute_name',axis=1)

X=data['input']
y= data.iloc[:,1:].values

X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.25,random_state=1)

vocab_size = 200

from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

max_length = 200

# X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
# X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# define rnn model

model = Sequential()
model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
model.add(Dense(100,activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

from keras.utils.vis_utils import plot_model

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
# print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)

# fit the model
model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)

occasion_pred_vectors_train = model.predict(X_train)
occasion_pred_vectors_test = model.predict(X_test)


In [948]:
threshold=0.15
occasion_train_pred_classes = pred_category(occasion_pred_vectors_train,threshold)
occasion_test_pred_classes = pred_category(occasion_pred_vectors_test,threshold)

occasion_train_pred_classes_name = pred_category_name(occasion_pred_vectors_train,threshold)
occasion_test_pred_classes_name = pred_category_name(occasion_pred_vectors_test,threshold)

occasion_train_true_classes = actual_category(y_train)
occasion_test_true_classes = actual_category(y_test)

occasion_train_true_classes_name = actual_category_name(y_train)
occasion_test_true_classes_name = actual_category_name(y_test)

occasion_test_F1=F1(occasion_test_pred_classes_name,occasion_test_true_classes_name)
occasion_train_F1=F1(occasion_train_pred_classes_name,occasion_train_true_classes_name)

In [949]:
occasion_train_F1

,attribute_name,attribute_value,recall,Precision,F1
0,occasion,coldweather,0.628458,1,0.771845
1,occasion,daytonight,0.987308,0.811321,0.890704
2,occasion,nightout,0.747259,0.944444,0.83436
3,occasion,vacation,0.698845,0.833333,0.760187
4,occasion,weekend,0.985863,0.927835,0.955969
5,occasion,work,0.848087,0.975,0.907126
6,occasion,workout,0.79661,1,0.886792


In [950]:
occasion_test_F1

,attribute_name,attribute_value,recall,Precision,F1
0,occasion,coldweather,0.265306,0,0
1,occasion,daytonight,0.911265,0.16,0.272206
2,occasion,nightout,0.539179,0.5,0.518851
3,occasion,vacation,0.483871,0,0
4,occasion,weekend,0.930956,0.347826,0.506436
5,occasion,work,0.683894,0.352941,0.465598
6,occasion,workout,0.647541,0.333333,0.440111


## subcategory_top

In [ ]:
attribute='subcategory_top'

data=df[df['attribute_name'] == attribute]
dummies = pd.get_dummies(data['attribute_value'])
data = pd.concat([data, dummies], axis=1, sort=False)
data=data.drop('attribute_value',axis=1)
data=data.drop('attribute_name',axis=1)

X=data['input']
y= data.iloc[:,1:].values

X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.25,random_state=1)

vocab_size = 200

from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

max_length = 200

# X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
# X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# define rnn model

model = Sequential()
model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
model.add(Dense(100,activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

from keras.utils.vis_utils import plot_model

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
# print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)

# fit the model
model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)

top_pred_vectors_train = model.predict(X_train)
top_pred_vectors_test = model.predict(X_test)


In [1162]:
threshold=0.15
top_train_pred_classes = pred_category(top_pred_vectors_train,threshold)
top_test_pred_classes = pred_category(top_pred_vectors_test,threshold)

top_train_pred_classes_name = pred_category_name(top_pred_vectors_train,threshold)
top_test_pred_classes_name = pred_category_name(top_pred_vectors_test,threshold)

top_train_true_classes = actual_category(y_train)
top_test_true_classes = actual_category(y_test)

top_train_true_classes_name = actual_category_name(y_train)
top_test_true_classes_name = actual_category_name(y_test)

top_test_F1=F1(top_test_pred_classes_name,top_test_true_classes_name)
top_train_F1=F1(top_train_pred_classes_name,top_train_true_classes_name)

In [ ]:
top_train_F1

In [ ]:
top_test_F1

## subcategory_bottom

In [753]:
attribute='subcategory_bottom'
data=df[df['attribute_name'] == attribute]
dummies = pd.get_dummies(data['attribute_value'])
data = pd.concat([data, dummies], axis=1, sort=False)
data=data.drop('attribute_value',axis=1)
data=data.drop('attribute_name',axis=1)

X=data['input']
y= data.iloc[:,1:].values

X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size=0.25,random_state=1)

vocab_size = 200

from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

max_length = 200

# X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
# X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# define rnn model

model = Sequential()
model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
model.add(Dense(100,activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

from keras.utils.vis_utils import plot_model

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
# print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)

# fit the model
model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)


In [754]:
threshold=0.15
bottom_train_pred_classes = pred_category(bottom_pred_vectors_train,threshold)
bottom_test_pred_classes = pred_category(bottom_pred_vectors_test,threshold)

bottom_train_pred_classes_name = pred_category_name(bottom_pred_vectors_train,threshold)
bottom_test_pred_classes_name = pred_category_name(bottom_pred_vectors_test,threshold)

bottom_train_true_classes = actual_category(y_train)
bottom_test_true_classes = actual_category(y_test)

bottom_train_true_classes_name = actual_category_name(y_train)
bottom_test_true_classes_name = actual_category_name(y_test)

bottom_test_F1=F1(bottom_test_pred_classes_name,bottom_test_true_classes_name)
bottom_train_F1=F1(bottom_train_pred_classes_name,bottom_train_true_classes_name)

In [863]:
bottom_test_F1

In [864]:
bottom_train_F1

In [870]:
F1(bottom_train_pred_classes_name,bottom_train_true_classes_name)

,attribute_name,attribute_value,recall,Precision,F1
0,subcategory_bottom,pants&leggings,1,0.960317,0.979757
1,subcategory_bottom,shorts,0.4,1,0.571429
2,subcategory_bottom,skirts,0.938776,1,0.968421


In [871]:
F1(bottom_test_pred_classes_name,bottom_test_true_classes_name)

,attribute_name,attribute_value,recall,Precision,F1
0,subcategory_bottom,pants&leggings,1,0.954545,0.976744
1,subcategory_bottom,shorts,0,,
2,subcategory_bottom,skirts,0.90625,1,0.95082


# RNN function

In [872]:
def run_rnn(attribute):
    data=df[df['attribute_name'] == attribute]
    dummies = pd.get_dummies(data['attribute_value'])
    data = pd.concat([data, dummies], axis=1, sort=False)
    data=data.drop('attribute_value',axis=1)
    data=data.drop('attribute_name',axis=1)

    X=data['text']
    y= data.iloc[:,1:].values

    X_train, X_test, y_train, y_test = \
    train_test_split(X,y, test_size=0.25,random_state=1)

    vocab_size = 200

    from keras.layers.recurrent import SimpleRNN
    from keras.layers import Flatten, Masking

    max_length = 200

    X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
    X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

    # define rnn model

    model = Sequential()
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    from keras.utils.vis_utils import plot_model

    # Compile the model
    model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    # print(model.summary())
    # plot_model(model, to_file='model.png', show_shapes=True)

    # fit the model
    model.fit(X_train, y_train,batch_size=200,epochs=50,verbose=0)
    
    pred_vectors_train = model.predict(X_train)
    pred_vectors_test = model.predict(X_test)
    
    train_pred_classes = pred_category(pred_vectors_train)
    test_pred_classes = pred_category(pred_vectors_test)

    train_pred_classes_name = pred_category_name(pred_vectors_train)
    test_pred_classes_name = pred_category_name(pred_vectors_test)
    
    train_true_classes = actual_category(y_train)
    test_true_classes = actual_category(y_test)

    train_true_classes_name = actual_category_name(y_train)
    test_true_classes_name = actual_category_name(y_test)
    
    test_F1=F1(test_pred_classes_name,test_true_classes_name)
    train_F1=F1(train_pred_classes_name,train_true_classes_name)
    
    return train_F1,test_F1
